# Fourierova spektroskopija

In [260]:
#Copyright (c) 2020 Tomaž Mertelj

%matplotlib inline
from ipywidgets import interact,interactive_output, HBox, VBox, Button, Output,Valid,BoundedFloatText
from ipywidgets import FloatSlider, Select, Dropdown, Layout,Label,HTML, Text, FloatLogSlider,GridspecLayout
import matplotlib.pyplot as plt
import numpy as np
import math

tau=0.004
noise=0.003
laser_k=2*math.pi/0.6328
Hg_lines=[[.4047,400],[.4358,1000],[.5462,500],[.5677105,200]]
Hg_cl=20
wl_k0=1/1.1
wl_kT=.2
wl_i0=500
i0=1
plt.rcParams["figure.figsize"] = (15, 5) # (w, h)
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.1, hspace=0.1)

def generate(src,len,step,apod,data_flag):
    x=np.arange(-len,len,step)
    freq=np.fft.fftfreq(x.size, d=2*step)
    inoise=np.random.normal(0*x,noise)
    if src=='Laser':
        idiode=i0*(np.cos(2*laser_k*x))
    elif src=='Hg':
        idiode=0*x
        for line in Hg_lines:
            k=2*math.pi/line[0]
            idiode=idiode+line[1]/1000*(np.cos(2*k*x)*np.exp(-(x/Hg_cl)**2))
    elif src=='Žarnica':
        spect=[]
        for fr in freq:
            af=abs(fr)
            if af>0:
                y=abs(af)**3*wl_i0/(np.exp(af/wl_kT)-1)/(1+np.exp(-50*(af-wl_k0)))*np.exp(4j*math.pi*fr*len)
            else:
                y=0
            spect.append(y)
        idiode=(np.fft.ifft(spect)).real
                    
    else:
        idiode=0*x
    idiode=idiode+inoise
    if apod=='Hamming':
        iapod=idiode*(0.54+0.46*np.cos(math.pi*x/len))
    elif apod=='Bartlet':
        iapod=idiode*(1-np.abs(x/len))
    elif apod=='Cosine':
        iapod=idiode*np.cos(math.pi*x/(2*len)) 
    elif apod=='Gaussian':
        iapod=idiode*np.exp(-2*(x/(len))**2)     
    else:
        iapod=idiode*1
    cspect=np.fft.fft(iapod)    
    spect=np.abs(cspect)
    coff=int(spect.size/2)
    spect=spect[:coff]
    freq=freq[:coff]
    fig, axes = plt.subplots(1,2)
    axes[0].plot(x, idiode)
    axes[0].plot(x, iapod)
    axes[1].plot(freq, spect)
    axes[0].set_ylabel(r'($I_{d}-I_{d0}$) (polj. en.)', fontsize=20)
    axes[1].set_ylabel(r'FFT($I_{d}$)', fontsize=20)
    axes[0].set_xlabel(r'$x$ ($\mu$m)', fontsize=20)    
    axes[1].set_xlabel(r'valovno št. (1/$\mu$m)', fontsize=20)
    axes[1].set_xlim([0,np.max(freq)])
    spect_widget.clear_output()
    ifg_widget.clear_output()
    if data_flag:
        fs_flag_widget.value=False
        with spect_widget:
            print('frekvenca\tamplituda')
            for idx, fi in np.ndenumerate(freq):
                print(f'{freq[idx]:.3}\t{spect[idx]}')
        with ifg_widget:
            print('pomik\tintenziteta\tapodizirano')
            for idx, id in np.ndenumerate(idiode):
                print(f'{x[idx]:.3}\t{idiode[idx]:.3}\t{iapod[idx]:.3}')    
    
scan_len_widget = BoundedFloatText(
    min=1, 
    max=1000, 
    step=1, 
    value=10, 
    continuous_update=False,description='Pomik zrcala ($\mu$m)',
    style={'description_width': '50%','width': '40%'}
)

scan_step_widget = BoundedFloatText(
    min=0.02, 
    max=0.3, 
    step=0.02, 
    value=.1, 
    continuous_update=False,description='Korak pomika ($\mu$m)',
    style={'description_width': '50%','width': '40%'}
)

source_widget =  Dropdown(
    options=['Laser','Hg','Žarnica'],
    value='Žarnica',
    description='Izvor',
    disabled=False,
#    layout=Layout(width='95%', height='auto'),     
    style={'description_width': '50%','width': '40%'}
)

apod_widget =  Dropdown(
    options=['Box','Bartlet','Cosine','Gaussian','Hamming'],
    value='Box',
    description='Apodizacija',
    disabled=False,
#    layout=Layout(width='95%', height='auto'),     
    style={'description_width': '50%','width': '40%'}
)

spect_widget=Output(
    layout={'border': '1px solid black', 'height': '300px', 'overflow_y': 'scroll'},
    description='Spekter:',
)

ifg_widget=Output(
    layout={'border': '1px solid black', 'height': '300px','overflow_y': 'scroll'},
    description='Interfrogram:',
)

fs_flag_widget=Valid(
    value=False,
    description='',
)

fs_widget=Button(
    description='Izpis podatkov',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Izpiše podatke',
    icon=''
)

def on_fs_widget_clicked(b):
    fs_flag_widget.value=True
    fs_widget.enabled=False


fs_widget.on_click(on_fs_widget_clicked)

out=interactive_output(generate, {'src':source_widget,
                                  'len':scan_len_widget, 
                                  'step':scan_step_widget, 
                                  'apod':apod_widget,
                                  'data_flag':fs_flag_widget
                                 });

params_box=VBox([HTML('<b> Parametri poskusa:</>'),source_widget,scan_len_widget,scan_step_widget,apod_widget])

grid=GridspecLayout(1,2)
grid[0,0]=ifg_widget
grid[0,1]=spect_widget

data_box=VBox([HTML('<b>Rezultati:</>'),grid])

display(params_box,out,fs_widget,data_box)

<Figure size 1080x360 with 0 Axes>

Output()

Button(description='Izpis podatkov', style=ButtonStyle(), tooltip='Izpiše podatke')